Design of a new game, Spuds

In [2]:
using Random
using DataFrames
using CSV
using Statistics
using Printf
using LinearAlgebra

In [11]:
struct Spud
    name::String
    h::Int16
    f::Int16
    l::Int16
    p::Int16
    r::Int16
    s::Int16
end

const MXS = 100
const MXV = 10
const MNV = 1

function cost(h::Int, f::Int, l::Int, p::Int, r::Int, s::Int)::Int
    return h*(f+l) + f*(p+r+s)
end

cost (generic function with 1 method)

In [20]:
function eval_battle(a::Spud, b::Spud)::Int
    a_finds = a.f >= b.h
    b_finds = b.f >= a.h
    score_a = 4 * (a.p > b.p) + 3 * (a.r > b.r) + 2 * (a.s > b.s)
    score_b = 4 * (a.p < b.p) + 3 * (a.r < b.r) + 2 * (a.s < b.s)
    wrestle_win = sign(score_a - score_b)
    if a_finds && b_finds
        tiebreaker = 4 * wrestle_win + 2 * sign(a.l - b.l) + sign(a.h - b.h)
        return sign(tiebreaker)
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        tiebreaker = 4 * wrestle_win + 2 * sign(a.l - b.l) + sign(a.f - b.f)
        return sign(tiebreaker)
    end
end


eval_battle (generic function with 1 method)

In [23]:
a = rand(library)
b = rand(library)
println(a)
println(b)
eval_battle(a,b)

Spud("Spud #10842", 10, 1, 7, 2, 10, 8)
Spud("Spud #10911", 10, 1, 8, 2, 6, 2)


1

In [38]:
tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)
condiments = CSV.read("condiments.csv", DataFrame)

function random_name_and_stat()::Spud
    vp = [0, 0, 0, 0, 0, 0]
    nametype = rand([1,1,1,1,1,2,2,2,3])
    name = ""
    noun = ""
    adj = ""
    if nametype == 1 || nametype == 2 || nametype == 3
        noun_i = rand(1:nrow(nouns))
        noun = nouns[noun_i, :noun]
        vp[1] = vp[1] + nouns[noun_i, :H]
        vp[2] = vp[2] + nouns[noun_i, :F]
        vp[3] = vp[3] + nouns[noun_i, :L]
        vp[4] = vp[4] + nouns[noun_i, :P]    
        vp[5] = vp[5] + nouns[noun_i, :R]
        vp[6] = vp[6] + nouns[noun_i, :S]    
    end
    if nametype == 1 || nametype == 3
        adj_i = rand(1:nrow(adjectives))
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H]
        vp[2] = vp[2] + adjectives[adj_i, :F]
        vp[3] = vp[3] + adjectives[adj_i, :L]
        vp[4] = vp[4] + adjectives[adj_i, :P]
        vp[5] = vp[5] + adjectives[adj_i, :R]
        vp[6] = vp[6] + adjectives[adj_i, :S]
    end
    if nametype == 2 || nametype == 3
        cond_i = rand(1:nrow(condiments))
        cond = condiments[cond_i, :condiment]
        vp[1] = vp[1] + condiments[cond_i, :H]
        vp[2] = vp[2] + condiments[cond_i, :F]
        vp[3] = vp[3] + condiments[cond_i, :L]
        vp[4] = vp[4] + condiments[cond_i, :P]
        vp[5] = vp[5] + condiments[cond_i, :R]
        vp[6] = vp[6] + condiments[cond_i, :S]
        name = string(noun, " with ", cond)
    end
    if nametype == 1
        name = string(adj, " ", noun)
    end
    if nametype == 2
        name = string(noun, " w/ ", cond)
    end
    if nametype == 3
        name = string(adj, " ", noun, " with ", cond)
    end
    Spud(name, vp[1], vp[2], vp[3], vp[4], vp[5], vp[6])
end

random_name_and_stat (generic function with 1 method)

In [55]:
function rand_rename(a::Spud, n_tries::Int = 100)::Spud
    best_score = 0.0
    best_b = random_name_and_stat()
    for ii in 1:n_tries
        b = random_name_and_stat()
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.p^2 + b.r^2 + b.s^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.p * b.p + a.r * b.r + a.s * b.s)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Spud(best_b.name, a.h, a.f, a.l, a.p, a.r, a.s)
end

rand_rename (generic function with 2 methods)

In [61]:
function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end


function pick_best(as::Array{Spud}, bs::Array{Spud})::Spud
    bestscore = -999
    bestf = as[1]
    for ii in 1:length(as)
        score = eval_battle_list(as[ii], bs)
        if score > bestscore
            bestscore = score
            bestf = as[ii]
        end
    end
    return bestf
end

function pick_best_rdmly(as::Array{Spud}, bs::Array{Spud}, ntries::Int)::Spud
    bestscore = -999
    bestf = rand(as)
    for ii in 1:ntries
        f = rand(as)
        score = eval_battle_list(f, bs)
        if score > bestscore
            bestscore = score
            bestf = f
        end
    end
    return bestf
end

function pick_best_rdmly_g(g::Function, bs::Array{Spud}, ntries::Int)::Spud
    bestscore = -999
    bestf = g()
    for ii in 1:ntries
        f = g()
        score = eval_battle_list(f, bs)
        if score > bestscore
            bestscore = score
            bestf = f
        end
    end
    return bestf
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, t = ts)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

function eval_team_battle(as::Array{Spud}, bs::Array{Spud})::Int
    a_i = 1
    b_i = 1
    while (a_i <= length(as)) && (b_i <= length(bs))
        res = eval_battle(as[a_i], bs[b_i])
        if res == 1
            b_i = b_i + 1
        else
            a_i = a_i + 1
            if res == 0
                b_i = b_i + 1
            end
        end
    end
    a_out = (a_i > length(as))
    b_out = (b_i > length(as))
    if a_out
        if b_out
            return 0
        else
            return -1
        end
    else
        return 1
    end
end

function compare_generator(f1, f2, limit)
    a_i = 1
    b_i = 1
    f_a = f1()
    f_b = f2()
    while (a_i < limit) && (b_i < limit)
        res = eval_battle(f_a, f_b)
        if res != -1
            b_i = b_i + 1
            f_b = f2()
        end
        if res != 1
            a_i = a_i + 1
            f_a = f1()
        end
    end
    return (a_i/limit, b_i/limit)
end

function random_team(f::Function, team_size::Int)::Array{Spud}
    team = Array{Spud}(undef, team_size)
    for i in 1:team_size
        team[i] = f()
    end
    return team
end

random_team (generic function with 1 method)

In [73]:
n_spuds = 11450 # determined after running this code
library = Array{Spud}(undef, n_spuds)
spud_i = 0
for h in MNV:MXV
    if (cost(h, MXV, MXV, MXV, MXV, MXV) >= MXS) && (cost(h, MNV, MNV, MNV, MNV, MNV) < MXS)
        for f in MNV:MXV
            if (cost(h, f, MXV, MXV, MXV, MXV) >= MXS) && (cost(h, f, MNV, MNV, MNV, MNV) < MXS)
                for l in MNV:MXV
                    if (cost(h, f, l, MXV, MXV, MXV) >= MXS) && (cost(h, f, l, MNV, MNV, MNV) < MXS)
                        for p in MNV:MXV
                            if (cost(h, f, l, p, MXV, MXV) >= MXS) && (cost(h, f, l, p, MNV, MNV) < MXS)
                                for r in MNV:MXV
                                    if (cost(h, f, l, p, r, MXV) >= MXS) && (cost(h, f, l, p, r, MNV) < MXS)
                                        for t in MNV:MXV
                                            if (cost(h,f,l,p,r,t) <= MXS)
                                                check_f = (h == MXV) || (cost(h+1,f,l,p,r,t) > MXS)
                                                check_h = (f == MXV) || (cost(h,f+1,l,p,r,t) > MXS)
                                                check_l = (l == MXV) || (cost(h,f,l+1,p,r,t) > MXS)
                                                check_prs = (p+r+t == 3*MXV) || (cost(h,f,l,p+1,r,t) > MXS)
                                                if check_h && check_f && check_l && check_prs
                                                    spud_i += 1
                                                    randname = rand_rename(Spud(" ",h,f,l,p,r,t)).name
                                                    name = string("#", @sprintf("%i", spud_i), ". ", randname)
                                                    library[spud_i] = Spud(name,h,f,l,p,r,t)
                                                end
                                            end
                                        end
                                    end
                                end                        
                            end
                        end                        
                    end
                end
            end
        end
    end
end
spud_i

11450

In [74]:
library

11450-element Vector{Spud}:
 Spud("#1. Dangerous Home Fries", 1, 3, 7, 10, 10, 10)
 Spud("#2. Battle Tater Tots", 1, 3, 10, 9, 10, 10)
 Spud("#3. Crazy Home Fries", 1, 3, 10, 10, 9, 10)
 Spud("#4. Hungry Home Fries with ketchup", 1, 3, 10, 10, 10, 9)
 Spud("#5. Hungry Tater Tots", 1, 4, 4, 3, 10, 10)
 Spud("#6. Hungry Chips", 1, 4, 4, 4, 9, 10)
 Spud("#7. Aquatic Home Fries", 1, 4, 4, 4, 10, 9)
 Spud("#8. Hungry Raw Potato", 1, 4, 4, 5, 8, 10)
 Spud("#9. Battle Tater Tots", 1, 4, 4, 5, 9, 9)
 Spud("#10. Friendly Home Fries", 1, 4, 4, 5, 10, 8)
 Spud("#11. Angry Tater Tots", 1, 4, 4, 6, 7, 10)
 Spud("#12. Battle Tater Tots", 1, 4, 4, 6, 8, 9)
 Spud("#13. Battle Tater Tots", 1, 4, 4, 6, 9, 8)
 ⋮
 Spud("#11439. Potato Soup w/ onion", 10, 5, 1, 5, 1, 2)
 Spud("#11440. Potato Soup w/ scallion", 10, 5, 2, 1, 1, 4)
 Spud("#11441. Potato Soup w/ scallion", 10, 5, 2, 1, 2, 3)
 Spud("#11442. Sneaky Mashed Potato", 10, 5, 2, 1, 3, 2)
 Spud("#11443. Devious Potato Soup", 10, 5, 2, 2, 1, 3)
 Spud("

## FP to compute Nash distribution

In [75]:
# Check that there are no ties in spudland
for i in 1:n_spuds
    for j in 1:n_spuds
        if i != j && eval_battle(library[i], library[j]) == 0
            println(library[i])
            println(library[j])
            println()
        end
    end
end

In [77]:
i_lose = Array{Int}(undef, (n_spuds, n_spuds))
n_lose = Array{Int}(undef, n_spuds)
for i in 1:n_spuds
    n_lose[i] = 0
    ff = library[i]
    for j in 1:n_spuds
        if eval_battle(ff, library[j]) ==-1
            n_lose[i] += 1
            i_lose[i, n_lose[i]] = j
        end
    end
end

In [78]:
# initialize FP with 1 of each
counts = [0 for i in 1:n_spuds]
wins = [0 for i in 1:n_spuds]
for i in 1:n_spuds
    counts[i] += 1
    for j in 1:n_lose[i]
        i_w = i_lose[i, j]
        wins[i_w]+= 1
    end
end

In [138]:
nits = 250000
for iter in 1:nits
    ind_winners = findall(wins .== maximum(wins))
    for i in ind_winners
        counts[i] += 1
        for j in 1:n_lose[i]
            i_w = i_lose[i, j]
            wins[i_w]+= 1
        end
    end
    if mod1(iter, 10000) == 1
        mxv, mxi = findmax(wins)
        print(library[mxi])
        print(" n:")
        print(counts[mxi])
        print(" w:")
        print(wins[mxi])
        println()
        mxv, mxi = findmax(counts)
        print(library[mxi])
        print(" n:")
        print(counts[mxi])
        print(" w:")
        print(wins[mxi])
        println()
        println()
    end
end

w = counts./sum(counts)
println(maximum([eval_battle_list2(ff, library, w) for ff in library]))
sum(counts)

Spud("#7940. Potato Strips w/ onion", 6, 10, 1, 1, 1, 1) n:12222 w:88587
Spud("#10824. Dangerous Potato Bread", 10, 1, 6, 10, 10, 10) n:20630 w:88585

Spud("#7940. Potato Strips w/ onion", 6, 10, 1, 1, 1, 1) n:12990 w:93229
Spud("#10824. Dangerous Potato Bread", 10, 1, 6, 10, 10, 10) n:21802 w:93226

Spud("#10824. Dangerous Potato Bread", 10, 1, 6, 10, 10, 10) n:22984 w:97879
Spud("#10824. Dangerous Potato Bread", 10, 1, 6, 10, 10, 10) n:22984 w:97879

Spud("#5004. Battle Chips", 5, 2, 6, 10, 10, 10) n:3107 w:102536
Spud("#10824. Dangerous Potato Bread", 10, 1, 6, 10, 10, 10) n:24173 w:102536

Spud("#9949. Potato Soup w/ onion", 8, 5, 1, 4, 2, 4) n:641 w:107185
Spud("#10824. Dangerous Potato Bread", 10, 1, 6, 10, 10, 10) n:25340 w:107178

0.10038873204751453


227406

In [133]:
# FP is slow to converge in the end
eps = 0.01/length(library)
nits = 20000
scores = wins./sum(counts)
w = counts./sum(counts)
0

0

In [135]:
previous_score = maximum(scores)
for iter in 1:nits
    ind_winners = findall(wins .== maximum(wins))
    i = rand(ind_winners)
    w[i] += eps
    for j in 1:n_lose[i]
        i_w = i_lose[i, j]
        scores[i_w]+= eps
    end
    if mod1(iter, 2000) == 1
        scores = scores./sum(w)
        w = w./sum(w)
        if maximum(scores) > previous_score
            eps = eps/2
            println(eps)
            println()
            println()
        end
        previous_score = maximum(scores)
        mxv, mxi = findmax(wins)
        print(library[mxi])
        print(" n:")
        print(w[mxi])
        print(" w:")
        print(scores[mxi])
        println()
        mxv, mxi = findmax(counts)
        print(library[mxi])
        print(" n:")
        print(w[mxi])
        print(" w:")
        print(scores[mxi])
        println()
        println(maximum([eval_battle_list2(ff, library, w) for ff in library]))
        println()
    end
end


Spud("#8260. Criminal Home Fries with onion", 7, 3, 1, 10, 10, 4) n:0.011885673609169086 w:0.4939630133566679
Spud("#10824. Dangerous Potato Bread", 10, 1, 6, 10, 10, 10) n:0.11444501336320011 w:0.4974163920328971
0.10927779742910886

8.528930131004367e-10


Spud("#8260. Criminal Home Fries with onion", 7, 3, 1, 10, 10, 4) n:0.011889044620889317 w:0.4939613281720038
Spud("#10824. Dangerous Potato Bread", 10, 1, 6, 10, 10, 10) n:0.11444462292712253 w:0.4974181066272091
0.10928083618170967

4.2644650655021836e-10


Spud("#8260. Criminal Home Fries with onion", 7, 3, 1, 10, 10, 4) n:0.011890730123874411 w:0.4939604855811126
Spud("#10824. Dangerous Potato Bread", 10, 1, 6, 10, 10, 10) n:0.11444442770941757 w:0.49741896392296187
0.10928235555542448

2.1322325327510918e-10


Spud("#8260. Criminal Home Fries with onion", 7, 3, 1, 10, 10, 4) n:0.011891572874648145 w:0.4939600642860248
Spud("#10824. Dangerous Potato Bread", 10, 1, 6, 10, 10, 10) n:0.11444433010064799 w:0.4974193925704156
0.1092